## 🤖 Warehouse RL Robot (BFS+RL Hybrid)


In [ ]:
!pip install gymnasium torch imageio numpy matplotlib -q

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from collections import deque
import os
import imageio

print("📦 Importing SB3...")
from stable_baselines3 import DQN
from stable_baselines3.her.her_replay_buffer import HerReplayBuffer

# ═══════════════════════════════════════════════════════════════════════════════
# 1. WAREHOUSE ENVIRONMENT
# ═══════════════════════════════════════════════════════════════════════════════

class WarehouseEnv(gym.Env):
    metadata = {"render_modes": ["rgb_array"], "render_fps": 10}

    def __init__(
        self,
        grid_size=20,
        obstacle_density=0.3,
        min_distance=None,
        max_distance=None,
        render_mode='rgb_array',
        fixed_start_goal=None,
    ):
        super().__init__()
        self.grid_size = grid_size
        self.max_steps = 200
        self.render_mode = render_mode
        self.obstacle_density = obstacle_density
        self.min_distance = min_distance if min_distance is not None else grid_size // 2
        self.max_distance = max_distance if max_distance is not None else np.sqrt(2) * grid_size
        self.fixed_start_goal = fixed_start_goal

        self.observation_space = spaces.Dict({
            'observation': spaces.Box(0, grid_size, shape=(2,), dtype=np.float32),
            'achieved_goal': spaces.Box(0, grid_size, shape=(2,), dtype=np.float32),
            'desired_goal': spaces.Box(0, grid_size, shape=(2,), dtype=np.float32),
        })
        self.action_space = spaces.Discrete(4)
        
        self._rng = np.random.default_rng()
        self.obstacles = None
        self._reset_world()

    def _sample_start_goal(self):
        if self.fixed_start_goal is not None:
            agent_pos, goal_pos = self.fixed_start_goal
            return np.array(agent_pos, dtype=np.int32), np.array(goal_pos, dtype=np.int32)

        for _ in range(500):
            agent = self._rng.integers(3, self.grid_size - 3, size=2, dtype=np.int32)
            goal = self._rng.integers(3, self.grid_size - 3, size=2, dtype=np.int32)
            dist = np.linalg.norm(goal - agent)
            if (
                dist >= self.min_distance
                and dist <= self.max_distance
                and not np.array_equal(agent, goal)
            ):
                return agent, goal

        return np.array([3, 3], dtype=np.int32), np.array([self.grid_size-4, self.grid_size-4], dtype=np.int32)

    def _reset_world(self):
        self.agent_pos, self.goal_pos = self._sample_start_goal()
        
        self.obstacles = np.zeros((self.grid_size, self.grid_size), dtype=np.int32)
        self.obstacles[0, :] = self.obstacles[-1, :] = 1
        self.obstacles[:, 0] = self.obstacles[:, -1] = 1
        
        if self.obstacle_density >= 0.20:
            col = self.grid_size // 2
            for row in range(3, self.grid_size-3):
                if row % 3 != 0:
                    self.obstacles[row, col] = 1
        
        if self.obstacle_density >= 0.35:
            for col in [self.grid_size // 3, 2 * self.grid_size // 3]:
                for row in range(3, self.grid_size-3):
                    if row % 3 != 1:
                        self.obstacles[row, col] = 1
        
        free_cells = np.argwhere(self.obstacles == 0)
        n_extra = int(len(free_cells) * self.obstacle_density * 0.4)
        if n_extra > 0:
            idx = self._rng.choice(len(free_cells), size=n_extra, replace=False)
            for (x, y) in free_cells[idx]:
                self.obstacles[x, y] = 1
        
        # FIXED: Larger safety zone (7x7)
        for dx in range(-3, 4):
            for dy in range(-3, 4):
                for pos in [self.agent_pos, self.goal_pos]:
                    p = pos + [dx, dy]
                    if 0 <= p[0] < self.grid_size and 0 <= p[1] < self.grid_size:
                        self.obstacles[tuple(p)] = 0
        
        self.steps = 0

    def reset(self, seed=None, options=None):
        if seed is not None:
            self._rng = np.random.default_rng(seed)
        self._reset_world()
        return self._get_obs(), {}

    def _get_obs(self):
        return {
            'observation': self.agent_pos.astype(np.float32),
            'achieved_goal': self.agent_pos.astype(np.float32),
            'desired_goal': self.goal_pos.astype(np.float32),
        }

    def step(self, action):
        self.steps += 1
        moves = [np.array([-1,0]), np.array([0,1]), np.array([1,0]), np.array([0,-1])]
        new_pos = self.agent_pos + moves[action]
        
        if (
            0 <= new_pos[0] < self.grid_size
            and 0 <= new_pos[1] < self.grid_size
            and self.obstacles[tuple(new_pos)] == 0
        ):
            self.agent_pos = new_pos
        
        obs = self._get_obs()
        reward = self.compute_reward(obs['achieved_goal'], obs['desired_goal'], None)
        terminated = (reward == 0.0)
        truncated = (self.steps >= self.max_steps)
        return obs, reward, terminated, truncated, {}

    def compute_reward(self, achieved_goal, desired_goal, info):
        if achieved_goal.ndim == 1:
            dist = np.linalg.norm(achieved_goal - desired_goal)
            return np.array(0.0 if dist < 1.0 else -1.0, dtype=np.float32)
        else:
            dist = np.linalg.norm(achieved_goal - desired_goal, axis=1)
            return np.array([0.0 if d < 1.0 else -1.0 for d in dist], dtype=np.float32)

    def render(self):
        if self.render_mode != 'rgb_array':
            return None
        size = 400
        img = np.ones((size, size, 3), dtype=np.uint8) * 240
        cell = size // self.grid_size
        
        for x in range(self.grid_size):
            for y in range(self.grid_size):
                if self.obstacles[x, y]:
                    img[x*cell:(x+1)*cell, y*cell:(y+1)*cell] = [139, 90, 43]
        
        gx, gy = self.goal_pos
        margin = cell // 4
        img[gx*cell+margin:(gx+1)*cell-margin, gy*cell+margin:(gy+1)*cell-margin] = [0, 200, 0]
        
        ax, ay = self.agent_pos
        img[ax*cell:(ax+1)*cell, ay*cell:(ay+1)*cell] = [30, 144, 255]
        return img

print("✅ WarehouseEnv loaded!")

# ═══════════════════════════════════════════════════════════════════════════════
# 2. BFS PATHFINDING
# ═══════════════════════════════════════════════════════════════════════════════

def bfs_search(grid_size, obstacles, start, goal):
    directions = [(-1,0), (0,1), (1,0), (0,-1)]
    queue = deque([(start, [])])
    visited = set([tuple(start)])
    
    while queue:
        pos, path = queue.popleft()
        if np.array_equal(pos, goal):
            return path + [pos]
        
        for dx, dy in directions:
            nx, ny = pos + np.array([dx, dy])
            if (
                0 <= nx < grid_size
                and 0 <= ny < grid_size
                and obstacles[int(nx), int(ny)] == 0
                and (int(nx), int(ny)) not in visited
            ):
                visited.add((int(nx), int(ny)))
                queue.append((np.array([nx, ny]), path + [pos]))
    return None

def get_action(current_pos, next_pos):
    dx, dy = next_pos - current_pos
    if dx == -1: return 0
    if dy == 1: return 1
    if dx == 1: return 2
    if dy == -1: return 3
    return 0

print("✅ BFS solver loaded!")

# ═══════════════════════════════════════════════════════════════════════════════
# 3. CURRICULUM LEARNING (SINGLE ENV - NO MULTIPROCESSING)
# ═══════════════════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("🎓 CURRICULUM LEARNING: EASY → MEDIUM → HARD")
print("="*70)

# Stage 1: EASY
print("\n📚 Stage 1/3: Training on EASY (density=0.10, distance=5-8)")
env_easy = WarehouseEnv(grid_size=20, obstacle_density=0.10, min_distance=5, max_distance=8)

model = DQN(
    "MultiInputPolicy",
    env_easy,
    replay_buffer_class=HerReplayBuffer,
    replay_buffer_kwargs=dict(n_sampled_goal=4, goal_selection_strategy='future'),
    learning_rate=1e-3,
    buffer_size=100000,
    learning_starts=2000,
    batch_size=256,
    verbose=1,
    device='cuda'
)

print("⏳ Training 100,000 steps on EASY...")
model.learn(total_timesteps=100000)
print("✅ Stage 1 complete!")

# Stage 2: MEDIUM (just update env parameters - no new env object)
print("\n📚 Stage 2/3: Training on MEDIUM (density=0.25, distance=8-12)")
env_easy.obstacle_density = 0.25
env_easy.min_distance = 8
env_easy.max_distance = 12

print("⏳ Training 100,000 steps on MEDIUM...")
model.learn(total_timesteps=100000, reset_num_timesteps=False)
print("✅ Stage 2 complete!")

# Stage 3: HARD
print("\n📚 Stage 3/3: Training on HARD (density=0.40, distance=12-16)")
env_easy.obstacle_density = 0.40
env_easy.min_distance = 12
env_easy.max_distance = 16

print("⏳ Training 100,000 steps on HARD...")
model.learn(total_timesteps=100000, reset_num_timesteps=False)
print("✅ Stage 3 complete!")

env_easy.close()

model.save("warehouse_dqn_her_curriculum")
print("\n🎉 CURRICULUM TRAINING COMPLETE! (300k total steps)")
print("⏱️  Estimated time: ~45-60 min on Kaggle T4")
print("="*70)

# ═══════════════════════════════════════════════════════════════════════════════
# 4. SIDE-BY-SIDE EVAL
# ═══════════════════════════════════════════════════════════════════════════════

def side_by_side_eval(model, num_runs=5):
    os.makedirs('videos_comparison', exist_ok=True)

    difficulties = {
        'tutorial': (20, 0.00, 3, 5),
        'easy':     (20, 0.10, 5, 8),
        'medium':   (20, 0.25, 8, 12),
        'hard':     (20, 0.40, 12, 16),
        'expert':   (20, 0.55, 14, 18),
        'nightmare':(20, 0.70, 16, 22),
    }

    results = {'bfs': [], 'rl': []}
    
    print("\n🎬 SIDE-BY-SIDE: BFS (BLUE) vs RL (GREEN)...")
    
    for diff_name, (size, density, min_dist, max_dist) in difficulties.items():
        print(f"\n🔄 {diff_name.upper()}")
        
        for run in range(1, num_runs + 1):
            tmp_env = WarehouseEnv(
                grid_size=size,
                obstacle_density=density,
                min_distance=min_dist,
                max_distance=max_dist,
            )
            obs_tmp, _ = tmp_env.reset()
            shared_start = tmp_env.agent_pos.copy()
            shared_goal = tmp_env.goal_pos.copy()
            tmp_env.close()

            fixed_pair = (shared_start, shared_goal)

            # BFS run
            env_bfs = WarehouseEnv(
                grid_size=size,
                obstacle_density=density,
                min_distance=min_dist,
                max_distance=max_dist,
                fixed_start_goal=fixed_pair,
            )
            obs_bfs, _ = env_bfs.reset()
            frames_bfs = [env_bfs.render()]

            path = bfs_search(env_bfs.grid_size, env_bfs.obstacles, obs_bfs['observation'], obs_bfs['desired_goal'])
            if path is not None:
                for next_pos in path[1:]:
                    action = get_action(obs_bfs['observation'], next_pos)
                    obs_bfs, _, term, trunc, _ = env_bfs.step(action)
                    frames_bfs.append(env_bfs.render())
                    if term or trunc:
                        break

            for _ in range(8):
                frames_bfs.append(env_bfs.render())
            steps_bfs = env_bfs.steps
            success_bfs = path is not None
            env_bfs.close()

            # RL run
            env_rl = WarehouseEnv(
                grid_size=size,
                obstacle_density=density,
                min_distance=min_dist,
                max_distance=max_dist,
                fixed_start_goal=fixed_pair,
            )
            obs_rl, _ = env_rl.reset()
            frames_rl = [env_rl.render()]
            last_reward = -1.0

            for _ in range(200):
                action, _ = model.predict(obs_rl, deterministic=True)
                obs_rl, reward, term, trunc, _ = env_rl.step(action)
                frames_rl.append(env_rl.render())
                last_reward = reward
                if term or trunc:
                    break

            for _ in range(8):
                frames_rl.append(frames_rl[-1])
            steps_rl = env_rl.steps
            success_rl = (last_reward == 0.0)
            env_rl.close()

            # Split-screen
            max_frames = max(len(frames_bfs), len(frames_rl))
            frames_bfs += [frames_bfs[-1]] * (max_frames - len(frames_bfs))
            frames_rl += [frames_rl[-1]] * (max_frames - len(frames_rl))

            split_frames = []
            for i in range(max_frames):
                left = frames_bfs[i].copy()
                right = frames_rl[i].copy()
                left[:, :, 2] = np.minimum(255, left[:, :, 2] + 50)
                right[:, :, 1] = np.minimum(255, right[:, :, 1] + 80)
                split_frames.append(np.hstack([left, right]))

            video_path = f'videos_comparison/{diff_name}_run{run:02d}_BFSvsRL.mp4'
            imageio.mimsave(video_path, split_frames, fps=10)
            print(
                f"   🎥 {diff_name} Run {run}: "
                f"BFS={steps_bfs}s {'✅' if success_bfs else '❌'} | "
                f"RL={steps_rl}s {'✅' if success_rl else '❌'} → {video_path}"
            )

            results['bfs'].append({'diff': diff_name, 'steps': steps_bfs, 'success': success_bfs})
            results['rl'].append({'diff': diff_name, 'steps': steps_rl, 'success': success_rl})
    
    print("\n🎯 Videos in /kaggle/working/videos_comparison/")
    return results

results = side_by_side_eval(model)

# Summary
print("\n" + "="*70)
print("📊 SUCCESS RATE SUMMARY")
print("="*70)
for diff in ['tutorial', 'easy', 'medium', 'hard', 'expert', 'nightmare']:
    bfs_success = sum(1 for r in results['bfs'] if r['diff'] == diff and r['success'])
    rl_success = sum(1 for r in results['rl'] if r['diff'] == diff and r['success'])
    print(f"{diff.upper():12s} | BFS: {bfs_success}/5 ({bfs_success*20}%) | RL: {rl_success}/5 ({rl_success*20}%)")

print("\n🚀 COMPLETE! 30 MP4s ready")
print("⏱️  Total time: ~45-60 min on Kaggle T4")
